In [1]:
# The data is using movie lens here, from 
## F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. 
## ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. 
## DOI=http://dx.doi.org/10.1145/2827872

import pandas as pd

In [2]:
root_folder = 'ml-100k/'
full_data = root_folder + 'u.data'

In [5]:
# The original data do not have column names in the csv file, you need to check their ReadMe, and add column names
full_data_cols = ['userid', 'itemid', 'rating', 'timestamp']
full_data_df = pd.read_csv(full_data, sep='\t', names = full_data_cols, encoding='latin-1')
full_data_df.head()

,userid,itemid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
